---
# Исследование надёжности заёмщиков

---

 

Заказчик — кредитный отдел банка. Входные данные — статистика о платёжеспособности клиентов.
Статистика представлена в виде таблицы, в файле формата csv. Содержит столбцы:

    - children — количество детей в семье
    - days_employed — общий трудовой стаж в днях
    - dob_years — возраст клиента в годах
    - education — уровень образования клиента
    - education_id — идентификатор уровня образования
    - family_status — семейное положение
    - family_status_id — идентификатор семейного положения
    - gender — пол клиента
    - income_type — тип занятости
    - debt — имел ли задолженность по возврату кредитов
    - total_income — ежемесячный доход
    - purpose — цель получения кредита

Результаты исследования будут учтены при построении модели **кредитного скоринга**.

В качестве цели анализа, необходимо ответить на следующие вопросы:
- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Как разные цели кредита влияют на его возврат в срок?


### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [20]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
print(data.tail(15))
print(data.info()) #изучаем общую информацию
print(data.isnull().sum()) # находим сумму неопределенных и пропущенных значений


       children  days_employed  dob_years            education  education_id  \
21510         2            NaN         28              среднее             1   
21511         0    -612.569129         29               высшее             0   
21512         0    -165.377752         26               высшее             0   
21513         0   -1166.216789         35              среднее             1   
21514         0    -280.469996         27  неоконченное высшее             2   
21515         1    -467.685130         28              среднее             1   
21516         0    -914.391429         42               высшее             0   
21517         0    -404.679034         42               высшее             0   
21518         0  373995.710838         59              СРЕДНЕЕ             1   
21519         1   -2351.431934         37       ученая степень             4   
21520         1   -4529.316663         43              среднее             1   
21521         0  343937.404131         6

### Вывод

По результатам проверки, обнаружено 2174 пропущенных значения в данных об общем трудовом стаже и ежемесячном доходе. Количество проопущеных значений совпадает.
Возможные причины пропусков: 
1. Некорректно предоставленные данные. 
2. Фактическое отсутствие трудового стажа и, соответственно, отсутствие ежемесячного дохода. 
3. Отсутствие справок с места работы, а, соответственно, отсутствие данных о стаже и ежемесячном доходе. 

Необходимо уточнить дополнительную информацию о принципе заполнения данных о трудовом стаже. 
В описании данных обозначено, что данные столбца исчисляются в днях, при этом в наличии:
1. отрицательные числа, 
2. данные типа float, что не подразумевает исчисления в целых днях,
3. данные о стаже, превышают возраст  заемщика. 



### Шаг 2. Предобработка данных

### Обработка пропусков

In [21]:
" Обработка пропусков в столбце total_income "
# Пропуски в столбце с ежемесячным уровнем дохода 
#могут быть связаны с отсутствием предоставленой информации о доходах. 
# Так как объем пропусков составляет 10% от всего массива, а другие 
#столбцы несут информацию важную для цели исследования, используем средний уровень дохода, относительно
# типов занятости для заполнения пропусков.
# Удаление строк может привести к искажению данных.

median_type_daysincome = data.groupby('income_type')['total_income'].median() #создаем новую таблицу, группируя
#тип занятости и находим средний ежемесячный доход. Далее переносим средние значения по типу занятости в 
#основную таблицу с помощью логической индексации
for row in median_type_daysincome.index:
    data.loc[data['income_type'] == row,'total_income']=\
    data.loc[data['income_type'] == row,'total_income'].fillna(median_type_daysincome.loc[row])

" Обработка пропусков в столбце days_employed "
# Пропуски в данном столбце совпадают с пропусками в total_income.
# Соответственно причина возникновения аналогична - отсутствие документа с места работы.
# Аналогично заполним пропуски средним значением по столбцу, чтобы не потерять информацию по другим показателям.
median_type_employed  = data.groupby('income_type')['days_employed'].median()
for row in median_type_employed.index:
    data.loc[data['income_type'] == row,'days_employed']=\
    data.loc[data['income_type'] == row,'days_employed'].fillna(median_type_employed.loc[row])
data.isnull().sum() # проверка заполнения

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Вывод

Количество дней стажа не влияют на цель исследования, в данных определенно присутствуют ошибки конвертации и для наиболее корректной обработки требуется дополнительная информация. Для того, чтобы не оставлять пропущеных значений, заполняем пропуски в соответствии со средним значением по типу занятости. 

Пропущенные значения в столбце  total_income заполняем пропуски в соответствии со средним значением по типу занятости, так как ежемесячный доход у типов занятости отличаются, и, в дальнейшем, эти данные будут влиять на итоги анализа. 

### Замена типа данных

In [22]:
# Столбцы days_employed и total_income имеют тип float24. 
# Вещественные числа не поддерживают длинную арифметику и усложняют читаемость. 
# Кроме этого, столбец days_employed должен нести информацию в целых числах, т.к. исчисление предполагается
#в днях. Для корректных расчетов при анализе и удобства сравнения, заменим тип на int. 

data['days_employed'] = data['days_employed'].astype('int') # замена типа данных
data['total_income'] = data['total_income'].astype('int')
data.info() # проверка


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

Все числовые данные приведены к единому формату. Очевидно, что при формировании таблицы, разработчики использовали определенную коныертацию, которая усложняет процесс предобработки данных и их анализа. Необходимо уточнение дополнительной информации о принципе формирования.

### Обработка дубликатов

In [23]:
"Проверка наличия дублей и некорректных значений в столбцах типа object"

data['education'].unique() #проверяем уникальные значения в столбце ОБРАЗОВАНИЕ
# Возвращаются одинаковые значения в разном регистре. 
data['education'] = data['education'].str.lower() # приводим значения столбца к нижнему ркгистру
data['education'].unique() # проверяем изменения
# Аналогично проверим столбцы остальнык столбцы формата object

data['family_status'].unique() #дубликаты отсутствуют

data['gender'].unique() # получаем значение XNA, которое не относится ни к одному полу.
XNA_found_count = data[data['gender'] == 'XNA']['gender'].count() # считаем колличество встречающихся значений
#XNA NA_found_count # 1 значение в таблице. Вероятно, ошибка ввода, либо заемщик не определился с половой 
#принадлежностью. Удалим строку с ошибкой 
df = data[data['gender'] != 'XNA'] #создание новой таблицы без артефакта
df['gender'].unique()# проверка. некорректные данные отсутствуют

df['income_type'].unique() #дубликаты отсутствуют

df['purpose'].unique() # много значений с одинаковым корнем. к дубликатам не относится, 
#но стоить применить лемматизацию в дальнейшей работе.

"Проверка на наличие дублей в строках"

df.duplicated().sum() # находим колличество дублированных строк
df = data.drop_duplicates().reset_index(drop=True) # удаляем дубликаты с сохранением порядка индексов
df.duplicated().sum() # проверяем изменения

0

### Вывод

В ходе предобработки данных на наличие дублей, были выявлены:

1. дубли в столбце "образование".
    Возможные причины появления:
        - Ручной, нерегламентированный ввод данных привел к отличиям в регистре.

2. некорректное значение в столбце "пол".
    Возможные причины появления:
        - Опечатка при ручном вводе данных в программу 
        - Заемщик относит себя к полу, отличному от общепринятых
    Так как это единственный случай, удаление данных о конкретном заемщике не создаст большой погрешности в выводах.


3. 71 дубликат строк. 
    Возможные причины появления:
        - Так как общий % дублей составляет менее 1% от общего массива, можем предположить, что была допущена ошибка при выгрузке данных разработчиками.
        - Заемщик оформлял заявку на кредит несколько раз и, вместо корректировки первой анкеты, завели вторую.

4. Цели получения кредита фиксируются в свободной форме text free, их можно разделить на леммы для дальнейшего анализа.




### Лемматизация

In [24]:
from pymystem3 import Mystem # импортируем pymystem3
m = Mystem()
df['purpose_lemm'] = df['purpose'].apply(lambda x: m.lemmatize(x)) # выводим список лемм для каждой строки 
# в отдельный столбец purpose_lemm

def purpose_type(row): # возвращаем основные группы с помощью условий
   
    if ('жилье' in row) or ('недвижимость' in row):
        return 'операции с недвижимостью'
    if 'свадьба' in row:
        return 'свадьба'
    if 'автомобиль' in row:
        return 'приобретение автомобиля'
    else:
        return 'образование'
    
df['purpose_group'] = df['purpose_lemm'].apply(purpose_type) # категоризируем с помощью функции
df['purpose_group'].unique() # проверка

array(['операции с недвижимостью', 'приобретение автомобиля',
       'образование', 'свадьба'], dtype=object)

### Вывод

Работать с единичными отрывками и делать из них статистические выводы нельзя. Значит, нужно сгруппировать данные, чтобы численности каждой группы хватало для формулировки выводов, необходимых заказчику. Исходя из целей исследования, необходимо учесть наиболее популярные категории. 
 Таким образом получили 4 основных группы целей для формирования выводов. 

### Категоризация данных

In [25]:
"Категоризация по наличию детей"

df['children'].value_counts() # проверим данные столбца, посчитав часто встречающиеся количества детей
# Находим 47 случаев с отрицательным значением. Возможные причины появления: некорректный ввод данных; наличие 
# совершеннолетних "детей"; указаны дети, которые фактически были, но, например, умерли. 
# Любая из причин возникновения, кроме ошибки, говорит о том, что детей нет. Отнесем отрицательные значения в 
# категорию "нет детей", количество таких значений не искозит итоговый вывод.

# Так как для вывода необходимо ответить на вопрос "есть ли зависимость между наличием детей..", разделим 
# данные на 2 категории.

def children_presence(children):
    #Возвращает категорию по принципу наличия детей:
       # - 'есть дети' при значении children > 0
       # - 'нет детей' при значении children <= 0 
    if children > 0:
        return 'есть дети'
    else:
        return 'нет детей'
children_presence(0)#проверка функции
df['children_id'] = df['children'].apply(children_presence) # создаем новый столбец с категориями, применяя
# функцию children_presence как аргумент к методу  apply 

"Категоризация по семейному статусу"
# Информация о семейном статусе разделена на два столбца. Вероятно family_status_id идентифицирует столбец 
#family_status проверим данные обоих столбцов
df['family_status'].value_counts() # в столбце встречаются 5 вариантов семеного статуса
df['family_status_id'].value_counts() # столбец полностью подтверждает отражение вышеуказанного столбца в 
#числовом формате
# Исходя из цели исследования, заказчика интересует как наличие супруга влияет на вероятность своевременной оплаты.
# Категоризируем семейные статусы отталкиваясь от этой логики. 
# Более удобный тип для дальнейшего анализа - object (не нужно дополнительно искать значение чисел). 
#Будем категоризировать столбец family_status

#Распределим заемщиков так:
#- женатики и сожители попадают в категорию "в браке", так как эти заемщики имеют постоянного партнера
#- остальные попадают в категорию "не в браке", так как партнера нет, либо не указывается

#сохраним столбец family_status (и покажем как умеем пользоваться методом replace)
df['family_status'] = df['family_status'].replace('женат / замужем', 'в браке')
df['family_status'] = df['family_status'].replace('гражданский брак', 'в браке')    
df['family_status'] = df['family_status'].replace('Не женат / не замужем', 'не в браке')
df['family_status'] = df['family_status'].replace('в разводе', 'не в браке')
df['family_status'] = df['family_status'].replace('вдовец / вдова', 'не в браке')
df['family_status'].value_counts() # проверка 

"Категоризация по уровню дохода"
#Для того, чтобы категоризировать показатель дохода, используем метод quantile для поиска средних показателей,
# относительно которых можно сформировать категории.
df['total_income'].quantile([.33, .66]).astype('int') 

#Распределим уровень дохода так:
# - 'ниже среднего', категория < 122162
# - 'средний', категория >= 122162 < 169339
# 'выше среднего' >= 169339

def total_income_lvl(total_income):
    if total_income < 122162:
        return 'ниже среднего'
    if (169339 > total_income >= 122162):
        return 'средний'
    else:
        return 'выше среднего'

df['income_lvl'] = df['total_income'].apply(total_income_lvl)  
df.head(10)

"Категоризация по наличию задолженности"

def check_debt(debt):
    if debt == 0:
        return 'нет долга'
    else:
        return 'есть долг'
df['debt_status'] = df['debt'].apply(check_debt)



### Вывод

В столбце о наличии детей предоставлены данные, корректность которых невозможно проверить, поэтому решение по их обработке принимается относительно поставленной задачи. 

Данные о семейном статусе изначально представлены в 5 вариантах. Тем не менее, ориентируясь на цель исследования, принято решение минимизировать формальность статусов в пользу фактического наличия супруга/ партнера. 

Данные об уровне дохода представляют собой не вполне реальные цифры. Необходимо дополнительное уточнение об источнике данных и принципе формирования. Возможна ошибка при конвертации.

Данные о наличии долга прдставлены в цифровом формате. Для осуществления дельнейшего анализа, упрощению чтения значений, принято решение переименовать и переформотировать данные.

Данные готовы для дальнейшего анализа и ответов на заданные вопросы. 

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [26]:
children_debt = df.pivot_table(index=['children_id'], columns = 'debt_status', values = 'debt', aggfunc='count')  
# создаем сводную таблицу с нужными данными 
children_debt['sum'] = children_debt.sum(axis=1) # суммируем строки по критерию кличества детей в новом столбце
children_debt['debt_returned'] = (children_debt['нет долга']/children_debt['sum'])*100 #определим % кредитов,
#выплаты по которым происхоодят в срок (по критерию количества детей)
children_debt

debt_status,есть долг,нет долга,sum,debt_returned
children_id,,,,
есть дети,677,6639,7316,90.746309
нет детей,1064,13074,14138,92.474183


### Вывод

- Есть ли зависимость между наличием детей и возвратом кредита в срок?
- Да, есть.

Люди без детей в 1,5 раза чаще обращаются за кредитом и являются более надежными заемщиками.
Наличие детей может стать фактором риска несвоевременной выплаты долга, т.к. данная категория граждан имеет более объемную финансовую нугрузку.
- Наличие задолженностей у клиентов с детьми 9.3%
- Наличие задолженностей у клиентов без детей 7.6%

Разница в 1.7% является достаточным основанием для повыщения %ставки, уменьшения кредитного лимита, увеличения сроков погашения, особые условия для страхования. 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [27]:
family_debt = df.pivot_table(index=['family_status'], columns = 'debt_status', values = 'debt', aggfunc='count')
family_debt['sum'] = family_debt.sum(axis=1) 
family_debt['debt_returned'] = (family_debt['нет долга']/family_debt['sum'])*100 
family_debt

debt_status,есть долг,нет долга,sum,debt_returned
family_status,,,,
в браке,1319,15171,16490,92.001213
не в браке,422,4542,4964,91.498791


### Вывод

- Есть ли зависимость между семейным положением и возвратом кредита в срок?
- Да, есть.

Люди, находящиеся в браке либо имеющие постоянного партнера, в 3 раза чаще обращаются за кредитом и являются более надежными заемщиками. Очевидно, что разница в количестве обращений связана с  социальными/ природными факторами. По данным ТАСС 62% населения страны находятся "в серьезных отношения", включая официальный брак. Данную категорию, так же, принятно считать более социально активной. С этим связаны и обращения за кредитами, в том числе для улучшения качества жизни.
- Наличие задолженностей у клиентов, состоящих в браке 8%
- Наличие задолженностей у клиентов без детей 8.6%
Разница в 0,6% может быть обусловлена более ответственным подходом как к жизни, так и к финансовым обязательствам. Данный вывод может стать основанием для повыщения %ставки, уменьшения кредитного лимита.


- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [28]:
income_debt = df.pivot_table(index=['income_lvl'], columns = 'debt_status', values = 'debt', aggfunc='count')  
income_debt['sum'] = income_debt.sum(axis=1)
income_debt['debt_returned'] = (income_debt['нет долга']/income_debt['sum'])*100 
income_debt

debt_status,есть долг,нет долга,sum,debt_returned
income_lvl,,,,
выше среднего,585,7212,7797,92.497114
ниже среднего,613,6851,7464,91.787245
средний,543,5650,6193,91.232036


### Вывод

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- Да, есть.


- Наличие задолженностей у клиентов, имеющих доход выше среднего 7,6%
- Наличие задолженностей у клиентов, имеющих  средний доход без детей 8.8%
- Наличие задолженностей у клиентов, имеющих доход ниже среднего 8,3%

Очевидно, что люди, имеющие доход выше среднего реже обращиются за кредитом и являются более надежными заемщиками. Для данной категории есть смысл предлагать повышенный кредитный лимит.

Разница между двумя другими категориями составляет 0,5%. При этом, люди с доходом среднего уровня  являются более надежными заемщиками, относительно заемщиков с уровнем ниже. Очевидно, что заемщики с уровнем дохода ниже среднего более рисковая категория из-за финансовых ограничений. Любые незапланированные расходы могут стать фактором риска для своевременного погашения кредита. Для данных категорий рекомендуется расширить возможность по увеличению срока погашения, увеличив %ставку, но уменьшив ежемесячный платеж с целью снижения рисков образования задолженности.



- Как разные цели кредита влияют на его возврат в срок?

In [29]:
purpose_debt = df.pivot_table(index=['purpose_group'], columns = 'debt_status', values = 'debt', aggfunc='count')  
purpose_debt['sum'] = purpose_debt.sum(axis=1)
purpose_debt['debt_returned'] = (purpose_debt['нет долга']/purpose_debt['sum'])*100 
purpose_debt

debt_status,есть долг,нет долга,sum,debt_returned
purpose_group,,,,
образование,370,3643,4013,90.779965
операции с недвижимостью,782,10029,10811,92.766627
приобретение автомобиля,403,3903,4306,90.640966
свадьба,186,2138,2324,91.996558


### Вывод

- Как разные цели кредита влияют на его возврат в срок?

- Операции с недвижимостью, наличие задолженности 7.3%
- Свадьба, наличие задолженности 8.1%
- Образование, наличие задолженности 9,3%
- Приобретение автомобиля, наличие задолженности 9.4%

Наиболее частая цель кредита - операции с недвижимостью. Категория заемщиков с данными целями является также наиболее надежной. Данный показатель объясняется одной из основных потребностей человека в безопасности и постоянным местом жительства. Задолженность по таким кредитам - слишком большой риск для заемщика. Данной категории рекомендуется увеличение кредитного лимита, расширение условий по страховке и оформления недвижимости в залог. 

Второй категорией по надежности является "Свадьба". Данная цель встречается реже всего. Высокий уровень возврата может быть связан с логичным делением финансовой нагрузки внутри вновь образовавшейся ячейки общества. Нельзя, также, исключать вероятность того, что заемщиком в данной категории может выступать не молодожен, а, например, родитель. Для данной категории заемщиков имеет смысл расширить возможности увеличения кредитного лимита,а так же - рассмотреть возможность предоставления спецусловий для следующих бОльших кредитных займов. Например, на приобритение жилья.

Следующие категории на 2% чаще допускают задолженности, относительно заемщиков с целями на операции с недвижимостью. Вероятно, заемщиками с целью "образование", являются люди с недостаточным уровнем квалификации либо его отсутствием. Возможна финансовая нестабильность. 
Причиной наличия долгов у категории "приобритение автомобиля" может быть связан с увеличением расходов заемщика после фактического осуществления цели. Факторы опыта вождения, состояния приобретаемого автомобиля - могут сказываться на наличии незапланированных расходов, увеличивая риск возникновения задолженности по кредиту. Данным категориям рекомендуется повышение %ставки, увеличения сроков погашения, особые условия для страхования, в случае "приобретения автомобиля" - оформление залога на автомобиль.


### Шаг 4. Общий вывод

Ответив на все поставленные вопросы, можно сделать следующие выводы:
1. Для оценки способности потенциального заёмщика вернуть кредит банку, необходимо учитывать факторы наличия детей, семейного положения, уровня дохода, цели кредита.
2. "Идеальный" заемщик: Лицо, состоящее в браке, не имеющее детей, с доходом выше среднего и целью проведения операций с недвижимостью.
3. Наиболее рисковый заемщик: не состоящий в браке, имеющий детей, доход ниже среднего и цель - приобретение автомобиля.
